In [ ]:
#https://wonhwa.tistory.com/16

In [ ]:
# url은 참고문서에서 따와야
# xml구조에 따라 명령어 달라짐. 봐야된다. 

In [60]:
# 모듈 import
import requests
import pprint

#인증키 입력
encoding = 'DspoWCubq1fqJqhtCsqFxS1EggF1ukemlgPgGaUSY%2F6K%2F1W0eCZguDUWDYbdb2bcbCF6TkStF2gPHE5GWE6wYA%3D%3D'
decoding = 'DspoWCubq1fqJqhtCsqFxS1EggF1ukemlgPgGaUSY/6K/1W0eCZguDUWDYbdb2bcbCF6TkStF2gPHE5GWE6wYA=='

#url 입력
url = 'http://apis.data.go.kr/6310000/ulsanmountain/getUlsanmountainList?serviceKey=DspoWCubq1fqJqhtCsqFxS1EggF1ukemlgPgGaUSY%2F6K%2F1W0eCZguDUWDYbdb2bcbCF6TkStF2gPHE5GWE6wYA%3D%3D&numOfRows=10&pageNo=1'
params ={'serviceKey' : 'decoding', 'pageNo' : '1', 'numOfRows' : '10', 'searchTitle' : '' }

response = requests.get(url, params=params)

# xml 내용
content = response.text

# 깔끔한 출력 위한 코드
pp = pprint.PrettyPrinter(indent=4)
#print(pp.pprint(content))

In [67]:
### xml을 DataFrame으로 변환하기 ###
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

## 각 컬럼 값 ## (포털 문서에서 꼭 확인하세요)
"""
resultCode:결과코드
resultMsg:결과메세지
pageIndex:쿼리페이지 시작점
numOfRows:페이지 크기
pageNo:시작 페이지
totalCount:전체 결과 수
unqId:고유번호
title:산명
intr:소개
roadNmAddr:소재지도로명주소
lotnoAddr:소재지지번주소
rprsTelno:연락처
hmpgAddr:홈페이지
lot:경도
lat:위도
cn:내용
addtCn:부가정보
mdfcnDt:데이터기준일자

""" 

#bs4 사용하여 item 태그 분리

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('list')
print(rows)

[<list><unqId>1</unqId><title>가지산</title><intr>봄이면 진달래, 여름이면 녹음, 가을이면 단풍, 겨울이면 눈으로 4계절 모두 비경을 연출하여 울산12경으로 지정</intr><roadNmAddr>울산광역시 울주군 상북면 / 경남 밀양시 산내면 / 경북 청도군 운문면</roadNmAddr><lotnoAddr>울산광역시 울주군 상북면 / 경남 밀양시 산내면 / 경북 청도군 운문면</lotnoAddr><rprsTelno>052-204-1727</rprsTelno><hmpgAddr/><lot>129.003367</lot><lat>35.620106</lat><cn>가지산은 영남알프스내의 가장 높은 산으로 1979년 경상남도 도립공원으로 지정되었다.&lt;br&gt;
울산 울주군, 경남 밀양시, 경북 청도군에 걸쳐 있으며, 해발 1,000m이상의 9개 고산으로 이어진 영남알프스의 주봉이다.&lt;br&gt;
주변의 귀바위, 무명봉인 1042봉, 1028봉, 1060봉 등이 가지산을 빙 둘러 대장처럼 호위하고 있다.&lt;br&gt;&lt;br&gt;
 
가지산 사계는 '울산12경'의 하나로 계절 따라 그 경관이 아름답기로 유명하다.&lt;br&gt;&lt;br&gt;

봄에는 진달래와 천연기념물 제462호인 철쭉군락지가 봄 정취를 한껏 자아내고 여름에는 석남사계곡, 심심이 계곡, 학소대폭포가 있는 학심이골, 오천평반석이 있는 쇠점골계곡, 호박소가 있는 용수골에서 더위를 피할 수 있다.&lt;br&gt;
시원한 계곡물에 발을 담그고 있으면 콧노래가 절로 나온다.&lt;br&gt;
가을에는 단풍이 곱게 물든 석남사계곡을 찾는 이가 많은데, 가을 풍광에 마음을 빼앗긴 사람들은 석남사를 그냥 지나치지 못한다.&lt;br&gt;
겨울에는 쌀바위 주변에 쌓인 설경이 아름답다.&lt;br&gt;
나무가지에 핀 상고대를 본 사람들은 "가지산은 겨울이 가장 아름답다."고 입을 모은다.&lt;br&gt;
가지산에서 발길을 돌리기 아쉽다면 가지산온천과 운문

In [68]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [70]:
#xml값 DataFrame으로 만들기
ulsan_mountain_df = pd.DataFrame(row_list, columns=name_list)
print(ulsan_mountain_df.head(19))

  unqId title                                               intr  \
0     1   가지산  봄이면 진달래, 여름이면 녹음, 가을이면 단풍, 겨울이면 눈으로 4계절 모두 비경을...   
1     2   간월산                                     신불산과 더불어 신성한 산   
2     7   고헌산        가뭄이 들면 산 정상에 있는 용샘에서 기우제를 지내기도 했던 곳으로 유명한 산   
3     8   대운산            울주군 온양읍 운화리에서 양산시 웅상읍 명곡리와 삼호리에 걸쳐 있는 산   
4    10   무룡산                                         용이 춤을 추던 산   
5    12   문복산                                           영남알프스 9봉   
6     9   문수산  문수보살이 머물렀다 하여 문수산이라 불리며 축선사, 망해사, 문수사, 영취사 등 4...   
7     3   신불산              가지산과 간월산에 이어져 영축산에 이르는 주봉으로 알려져 있는 고산   
8     4   영축산         기암괴석과 노송, 신불산 정상으로 이어지는 억새능선이 장관을 이루고 있는 산   
9    11   운문산                    영남알프스산군 중 경관이 가장 아름답다고 알려져 있는 산   

                                roadNmAddr  \
0  울산광역시 울주군 상북면 / 경남 밀양시 산내면 / 경북 청도군 운문면   
1               울산광역시 울주군 상북면 이천리 / 등억알프스리   
2     울산광역시 울주군 언양읍, 상북면, 두서면 / 경북 경주시 산내면   
3               울산광역시 울주군 온양읍 / 경남 양산시 웅상읍   
4                    

In [72]:
ulsan_mountain_df.to_csv('ulsan_mountain.csv', encoding='utf-8')